In [5]:
# KBO 일정 크롤링
## 2024 UPDATE - 기존에 사용하던 BS 와 다르게 경기 일정은 JS 를 통해 동적으로 생성되기 때문에 Selenium 사용 필요해짐

### 1. 패키지 세팅 
from selenium import webdriver
# from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup



In [5]:
## TEST
# options = Options()
# options.binary_location = "/usr/local/bin/chromedriver"
# driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome()

url = "https://m.sports.naver.com/kbaseball/schedule/index?date=2024-04-30"
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
contents = soup.find_all("li", {"class": "MatchBox_match_item__3_D0Q"})

contentArray = [];

for i in contents:
    item = i.find("div", {"class": "MatchBox_match_area__39dEr"})
    match = item.find_all("div", {"class": "MatchBoxTeamArea_name_info__2IaZV"})
    info = []
    
    for j in match:
        info.append(j.find("strong", {"class": "MatchBoxTeamArea_team__3aB4O"}).string)
    

    if len(info) < 1:
        break;

    contentArray.append({"home": info[1], "away": info[0]})

print(contentArray) #배열길이가 0이면 경기 없음

[{'home': 'KIA', 'away': 'KT'}, {'home': 'NC', 'away': 'LG'}, {'home': '한화', 'away': 'SSG'}, {'home': '두산', 'away': '삼성'}, {'home': '롯데', 'away': '키움'}]


In [27]:
## 개막 날짜 넣어서 모든 경기 일정 불러오기
from datetime import date, timedelta

startDateString = '2024-03-24'
startDate = date.fromisoformat(startDateString)
endDateString = '2024-12-01' # 임의의 종료 날짜
endDate = date.fromisoformat(endDateString)

dates = [];
dateDiff = date(endDate.year, endDate.month, endDate.day) - date(startDate.year, startDate.month, startDate.day)

for i in range(0, dateDiff.days):
    dates.append((date.fromisoformat(startDateString) + timedelta(days = i)).isoformat())

print(dates)

['2024-03-24', '2024-03-25', '2024-03-26', '2024-03-27', '2024-03-28', '2024-03-29', '2024-03-30', '2024-03-31', '2024-04-01', '2024-04-02', '2024-04-03', '2024-04-04', '2024-04-05', '2024-04-06', '2024-04-07', '2024-04-08', '2024-04-09', '2024-04-10', '2024-04-11', '2024-04-12', '2024-04-13', '2024-04-14', '2024-04-15', '2024-04-16', '2024-04-17', '2024-04-18', '2024-04-19', '2024-04-20', '2024-04-21', '2024-04-22', '2024-04-23', '2024-04-24', '2024-04-25', '2024-04-26', '2024-04-27', '2024-04-28', '2024-04-29', '2024-04-30', '2024-05-01', '2024-05-02', '2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06', '2024-05-07', '2024-05-08', '2024-05-09', '2024-05-10', '2024-05-11', '2024-05-12', '2024-05-13', '2024-05-14', '2024-05-15', '2024-05-16', '2024-05-17', '2024-05-18', '2024-05-19', '2024-05-20', '2024-05-21', '2024-05-22', '2024-05-23', '2024-05-24', '2024-05-25', '2024-05-26', '2024-05-27', '2024-05-28', '2024-05-29', '2024-05-30', '2024-05-31', '2024-06-01', '2024-06-02', '2024

In [ ]:
driver = webdriver.Chrome()

baseUrl = "https://m.sports.naver.com/kbaseball/schedule/index?date="

data = {}

for d in dates:
    matchesByDay = []
    url = ''.join([baseUrl, d])
    driver.get(url)
    
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('css selector', '.ScheduleAllType_match_list__3n5L_')))
        matches = element.find_elements('css selector', '.MatchBox_item_content__3SGZf')
        
        for el in matches:
            names = el.find_elements('css selector', '.MatchBoxTeamArea_team__3aB4O')
            matchesByDay.append({
                'home': names[1].text,
                'away': names[0].text
            })
    except:
#         경기가 없는 날엔 timeout 으로 exception 에 들어옵니다
#         print('경기가 없습니다')
        continue
        
    data[d] = matchesByDay

print(data)
driver.quit()


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 52 kB 4.6 MB/s eta 0:00:011
     |████████████████████████████████| 202 kB 4.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
